# Python으로 웹 스크래퍼 만들기
> 강의 링크 : https://academy.nomadcoders.co/courses/enrolled/681401
- URL 구조 분해
- HTML 구조 달라질 수 있음에 유의
- 코드 정리하며 스크레이핑하는 일련의 과정을 함수로 빌드업하는 방법
- 항상 빈 리스트 반환, 출력 등의 방법을 통해 중간 점검


# 2.1. What are We Building?

* 파이썬 관련 일자리를 구한다고 가정.
* `Indeed`(https://www.indeed.jobs/), `StackOverflow`(https://stackoverflow.com/) 사이트에서 파이썬 구인 공고 검색.
    - 직무
    - 회사 이름
    - 지원 링크
* 스크레이퍼를 이용해 두 사이트의 모든 페이지를 들어가 등록된 구인 공고를 가져온다.

## 2.2. Navigating with Python

### [`Requests`](https://requests.readthedocs.io/en/master/)
* 파이썬에서 url에 요청 보내고 정보 받음.

### [`BeautifulSoup`](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
* html에서 정보를 추출하는 파이썬 라이브러리.

In [1]:
# module import
import requests
from bs4 import BeautifulSoup

# 사이트 GET 요청으로 접속
indeed_result= requests.get("https://www.indeed.com/jobs?q=python&limit=50")
print(indeed_result) # 접속 성공 : <Response [200]>

# html 구조 살펴보기
indeed_result.text # html 정보를 모두 가져옴.

<Response [200]>


'<!DOCTYPE html>\n<html lang="en" dir="ltr">\n<head>\n<meta http-equiv="content-type" content="text/html;charset=UTF-8">\n<script type="text/javascript" src="/s/ea358df/en_US.js"></script>\n<link href="/s/e1d8b3d/jobsearch_all.css" rel="stylesheet" type="text/css">\n<link rel="alternate" type="application/rss+xml" title="Python Jobs, Employment" href="http://rss.indeed.com/rss?q=python">\n<link rel="alternate" media="only screen and (max-width: 640px)" href="/m/jobs?q=python&limit=50">\n<link rel="alternate" media="handheld" href="/m/jobs?q=python&limit=50">\n\n<script type="text/javascript">\n\nif (typeof window[\'closureReadyCallbacks\'] == \'undefined\') {\nwindow[\'closureReadyCallbacks\'] = [];\n}\n\nfunction call_when_jsall_loaded(cb) {\nif (window[\'closureReady\']) {\ncb();\n} else {\nwindow[\'closureReadyCallbacks\'].push(cb);\n}\n}\n</script>\n<meta name="ppstriptst" content="1">\n<script>\nvar _scriptDownloadCount = 0;\nvar _retryDownload = function() {\nvar script = documen

## 2.3. Extracting Indeed Pages

* `BeautifulSoup` 객체 생성
* `find_all` : 해당하는 요소 모두 찾아 리스트로 반환

### `Indeed` 사이트 스크레이핑 

1. url
> url : https://www.indeed.com/jobs?q=python&limit=50
    * 검색어 : Python
    * 검색 결과 : 50개 보여주도록 한정

2. 페이지 번호
    * BeautifulSoup을 이용해서 페이지 번호를 가져 온다.

In [2]:
from bs4 import BeautifulSoup

# html에서 정보 추출
indeed_soup = BeautifulSoup(indeed_result.text, 'html.parser') # html.parser 이용

# 페이지 번호 추출 : 마지막 항목 제외 모든 페이지 번호 다 가져오기.
pagination = indeed_soup.find("div", {"class" : "pagination"}) # 페이지 번호는 div 태그 아래 a에 있음.
pages = pagination.find_all("a") # a 태그 아래 span을 찾아야 함.
spans = []
for page in pages:
    # print(page.find("span")) # 오케이! 정보 가져온다. 나중에 가장 높은 숫자 찾는 함수 만들 것.
    spans.append(page.find("span")) 
print(spans[:-1]) # Next 제거. 

[<span class="pn">2</span>, <span class="pn">3</span>, <span class="pn">4</span>, <span class="pn">5</span>, <span class="pn">6</span>, <span class="pn">7</span>, <span class="pn">8</span>, <span class="pn">9</span>, <span class="pn">10</span>, <span class="pn">11</span>, <span class="pn">12</span>, <span class="pn">13</span>, <span class="pn">14</span>, <span class="pn">15</span>, <span class="pn">16</span>, <span class="pn">17</span>, <span class="pn">18</span>, <span class="pn">19</span>, <span class="pn">20</span>]


## 2.4. Extracting Indeed Pages 2

* anchor 요소 안에 다른 요소가 있고, 태그 정보 외에 string이 하나만 있기 때문에 `.string` 메서드 바로 사용.
* `Indeed` 홈페이지에서 가장 큰 페이지 숫자 반환.

In [3]:
# 페이지 번호 추출, 페이지 번호 정수로 변환
links = pagination.find_all("a")
pages = []
for link in links:
    # pages.append(link.find("span").string) # string만 가져온다.
    pages.append(link.string) # 같은 결과, 짧은 코드.
pages = pages[:-1]

# 페이지 번호 정수로 변환 : Next 때문에 오류가 생기므로 다른 방법 사용
links = pagination.find_all("a")
pages = []
for link in links[:-1]:
    pages.append(int(link.string))
print(pages)

# 가장 큰 숫자
max_page = pages[-1]
print(max_page)

# 페이지를 계속해서 request하는 방법
# URL에서의 start num : page number * 50


[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
20


## 2.5. Requesting Each Page

* 각 페이지를 계속해서 request한다. = 최대 페이지 수만큼 request 요청 보냄.
* 함수 생성 : 재사용 가능

    - `indeed.py` : 사용할 함수 작성
    - `main.py` : 
    ```
    from indeed import extract_indeed_pages
    from indeed import extract_indeed_jobs
    ```


In [4]:
# max_page를 이용해 숫자 배열 생성 : range의 현재 값에 indeed 요소 수를 곱함.
for n in range(max_page):
    print(f"start={n*50}") # url에서 start 이후 부분 만들 수 있음. "요청해야 할 값!"

start=0
start=50
start=100
start=150
start=200
start=250
start=300
start=350
start=400
start=450
start=500
start=550
start=600
start=650
start=700
start=750
start=800
start=850
start=900
start=950


In [5]:
# module import
import requests
from bs4 import BeautifulSoup

# 변수
LIMIT = 50 # 게시물 몇 개씩 표시할 것인지 변경 가능
URL = f"https://www.indeed.com/jobs?q=python&limit={LIMIT}"

# "indeed.py" : 가장 큰 검색 결과 페이지 반환
def extract_indeed_pages():
    result = requests.get(URL)
    soup = BeautifulSoup(result.text, 'html.parser')
    pagination = soup.find("div", {"class" : "pagination"})
    
    links = pagination.find_all("a")
    pages = []
    for link in links[:-1]:
        pages.append(int(link.string))
    
    max_page = pages[-1]
    return max_page

# 마지막 페이지까지 얻고 요청 보내는 함수
def extract_indeed_jobs(last_page):
    for page in range(last_page):
        # print(f"&start={page*LIMIT}")  
        result = requests.get(f"{URL}&start={page*LIMIT}")
        print(result.status_code) # 접속 상태 확인

# main.py
last_indeed_page = extract_indeed_pages()
extract_indeed_jobs(last_indeed_page) # 마지막 페이지 받아서 원활하게 접속됨.

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


## 2.6. Extracting Titles

* extract_indeed_jobs 수정
    * 각 페이지에서 일자리 정보 추출
    * 모든 일자리 추출 후 jobs 리스트로 반환
    
*cf) 내가 사용한 방법) print(result.find("div", {"class" : "title"}).text.strip('\n'))*

* 일자리 title을 찾기 위해 개발자 도구 사용
    * class가 `jobsearch...` 인 `div` 태그.
    

In [6]:
# jobs 리스트 반환하는 함수
def extract_indeed_jobs(last_page):
    jobs = []
    req = requests.get(f"{URL}&start={0*LIMIT}") # 첫 페이지 대상 테스트
    soup = BeautifulSoup(req.text, 'html.parser')
    results = soup.find_all("div", {"class" : "jobsearch-SerpJobCard"}) # job 정보 담고 있는 태그
    for result in results:
        title = result.find("div", {"class" : "title"}).find("a")["title"]
        print(title)
    return jobs

extract_indeed_jobs(last_indeed_page)

Jr. Python developer
Data Scientist - Entry Level
Content Contributor: Python and Flask
Python Developer Intern
Entry Level Data Analyst
Software Engineer - Python / Django - Telecommute
Full Time Opportunities for Students or Recent Graduates: Data & Applied Sciences- Atlanta, GA
Senior Full Stack Developer
Jr. Back-end Developer, Python
Python Developer
Freelance Web Developer
Laboratory Reporting Epidemiologist
Python Developer
Python / Kubernetes Developer
Back End Developer (Python)
Software Engineer (Python) (Remote position)
Python developer @ Princeton, NJ
Software Developer - Python, Linux, Shell Scripting (Work Remotely)
Onsite Positions
Junior Python Developer
Front end developer (Python)
Python Developer
Entry Level Biostatistician
Research Fellow
Python/SQL Developer
Python Developer
Research Scientist
Python/MongoDB Developer
Full-Time Research Assistant
Jr. Python Developer
Quantitative Trader - Entry Level
Software Engineer
Python Developer
Data Analyst
Online Developer

[]

## 2.7. Extracting Companies

* inspect pages : class가 company인 span
* 문제 : 회사 링크가 없는 경우가 있다! -> 이 경우 a 태그가 없다.

In [7]:
# 함수 수정 : 회사명까지 가져옴.
def extract_indeed_jobs(last_page):
    jobs = []
    req = requests.get(f"{URL}&start={0*LIMIT}")
    soup = BeautifulSoup(req.text, 'html.parser')
    results = soup.find_all("div", {"class" : "jobsearch-SerpJobCard"})
    
    for result in results:
        title = result.find("div", {"class" : "title"}).find("a")["title"]
        company = result.find("span", {"class" : "company"})
        company_anchor = company.find("a")
        if company_anchor is not None: # 링크가 있는 경우 a 태그 아래 문자열.
            company = str(company_anchor.string) # 문자열로 변환.
        else: # 링크 없으면 span의 문자열
            company = str(company.string)
        company = company.strip() # 공백 제거
        print(title, company) # 테스트용
    return jobs

extract_indeed_jobs(last_indeed_page)

Jr. Python developer Ace-stack LLC
Data Scientist - Entry Level Pathrise
Content Contributor: Python and Flask Codecademy
Python Developer Intern OXO Solutions
Entry Level Data Analyst Hopjump
Software Engineer - Python / Django - Telecommute Ellumen, Inc
Full Time Opportunities for Students or Recent Graduates: Data & Applied Sciences- Atlanta, GA Microsoft
Senior Full Stack Developer Cloud Haven Solutions
Jr. Back-end Developer, Python Boxy Charm
Python Developer Callsign Media
Freelance Web Developer RedRover Sales & Marketing Strategy
Laboratory Reporting Epidemiologist Dept of State Health Services
Python Developer Sreyo Inc
Python / Kubernetes Developer Applied Memetics LLC
Back End Developer (Python) Ace-stack LLC
Software Engineer (Python) (Remote position) Catasys
Python developer @ Princeton, NJ CEDENT
Software Developer - Python, Linux, Shell Scripting (Work Remotely) Capgemini
Onsite Positions Lumium
Junior Python Developer INTELLECTYX DATA SCIENCE INDIA
Front end developer

[]

## 2.8. Extracting Locations and Finishing up
* 정보 추출하는 부분을 빼서 따로 함수로 만들어 준다.
* location 추출
    - 강의에서의 문제 : 어떤 일자리는 location이 없다. class가 location인 span을 그냥 읽어오면 안 된다.
    - 강의에서의 대안 : class가 recJobLoc인 div 태그를 읽어 온다. `display : None`이어서 웹 화면에서 숨겨진다. 그렇지만 위치 정보를 가지고 있다. 위치 정보는 해당 태그에서 `data-rc-loc`이라는 속성에 존재한다.
* 지원하기 링크 추출 : 지원하기 링크로 이동하려면 url 뒤에 붙는 job-id가 필요하다.
    - class가 `jobsearch-SerpJobCard`인 div : 원래 추출해 놨던 result
    - `data-jk`라는 속성에 job id 있음.
    
* 강의에서는 오류 없었지만, 지금 내가 코드 작성하는 시점에는 9페이지 첫 게시물에 company가 없다!

In [8]:
# html 페이지에서 정보 추출하는 함수. dict 반환.
def extract_job(html):
    # 직무
    title = html.find("div", {"class" : "title"}).find("a")["title"]
    # 회사명 : 강의 코드 수정
    try:
        company = html.find("span", {"class" : "company"})
        company_anchor = company.find("a")
        if company_anchor is not None:
            company = str(company_anchor.string)
        else:
            company = str(company.string)
        company = company.strip()
    except AttributeError as e:
        print(f"회사 정보가 없는 공고가 있습니다.")
        company = "No Information"        
    # 위치
    location = html.find("div", {"class" : "recJobLoc"})["data-rc-loc"]
    # 지원하기 링크
    job_id = html["data-jk"] # job-id : 지원하기 링크로 이동할 수 있는 정보
    link = f"https://www.indeed.com/viewjob?jk={job_id}" # indeed.com에서 jk 다음에 id.
    return {'title' : title, 'company' : company, 'location' : location, "link" : link}

# 최종적으로 모든 페이지에 요청을 보내며 정보 추출
def extract_indeed_jobs(last_page):
    jobs = []
    for page in range(last_page):  
        print(f"Scrapping page {page+1}") # 확인용
        req = requests.get(f"{URL}&start={page*LIMIT}")
        soup = BeautifulSoup(req.text, 'html.parser')
        results = soup.find_all("div", {"class" : "jobsearch-SerpJobCard"})
        for result in results: 
            job = extract_job(result)
            jobs.append(job)
    return jobs
   
# main.py
last_indeed_page = extract_indeed_pages()
indeed_jobs = extract_indeed_jobs(last_indeed_page)
print(indeed_jobs)

Scrapping page 1
Scrapping page 2
Scrapping page 3
Scrapping page 4
Scrapping page 5
Scrapping page 6
Scrapping page 7
회사 정보가 없는 공고가 있습니다.
Scrapping page 8
회사 정보가 없는 공고가 있습니다.
Scrapping page 9
회사 정보가 없는 공고가 있습니다.
Scrapping page 10
Scrapping page 11
Scrapping page 12
Scrapping page 13
Scrapping page 14
Scrapping page 15
회사 정보가 없는 공고가 있습니다.
Scrapping page 16
Scrapping page 17
회사 정보가 없는 공고가 있습니다.
Scrapping page 18
Scrapping page 19
Scrapping page 20
회사 정보가 없는 공고가 있습니다.
[{'title': 'Jr. Python developer', 'company': 'Ace-stack LLC', 'location': 'Frisco, TX', 'link': 'https://www.indeed.com/viewjob?jk=7d956d35ccf949e0'}, {'title': 'Data Scientist - Entry Level', 'company': 'Pathrise', 'location': 'Remote', 'link': 'https://www.indeed.com/viewjob?jk=602196c5ea2f32da'}, {'title': 'Content Contributor: Python and Flask', 'company': 'Codecademy', 'location': 'Remote', 'link': 'https://www.indeed.com/viewjob?jk=eeeeaca1d923b164'}, {'title': 'Python Developer Intern', 'company': 'OXO Solutions', '

## 최종 정리

### `Indeed.py`

In [9]:
# module import
import requests
from bs4 import BeautifulSoup

# 변수
LIMIT = 50 # 게시물 몇 개씩 표시할 것인지 변경 가능
URL = f"https://www.indeed.com/jobs?q=python&limit={LIMIT}"

# 마지막 페이지 호출
def get_last_page():
    result = requests.get(URL)
    soup = BeautifulSoup(result.text, 'html.parser')
    pagination = soup.find("div", {"class" : "pagination"})
    
    links = pagination.find_all("a")
    pages = []
    for link in links[:-1]:
        pages.append(int(link.string))
    
    max_page = pages[-1]
    return max_page


# html 페이지에서 정보 추출하는 함수. dict 반환.
def extract_job(html):
    # 직무
    title = html.find("div", {"class" : "title"}).find("a")["title"]
    # 회사명
    try:
        company = html.find("span", {"class" : "company"})
        company_anchor = company.find("a")
        if company_anchor is not None:
            company = str(company_anchor.string)
        else:
            company = str(company.string)
        company = company.strip()
    except AttributeError as e:
        print(f"회사명이 없는 공고가 있습니다.")
        company = "No Information"
    # 위치
    location = html.find("div", {"class" : "recJobLoc"})["data-rc-loc"]
    # 지원하기 링크
    job_id = html["data-jk"] 
    link = f"https://www.indeed.com/viewjob?jk={job_id}" # indeed.com에서 jk 다음에 id.
    return {'title' : title, 'company' : company, 'location' : location, "link" : link}

# 모든 페이지 스크레이핑
def extract_jobs(last_page):
    jobs = []
    for page in range(last_page):  
        print(f"Scrapping page {page+1}") # 확인용
        req = requests.get(f"{URL}&start={page*LIMIT}")
        soup = BeautifulSoup(req.text, 'html.parser')
        results = soup.find_all("div", {"class" : "jobsearch-SerpJobCard"})
        for result in results: 
            job = extract_job(result)
            jobs.append(job)
    return jobs
   
# 모든 함수
def get_jobs():
    last_page = get_last_page()
    jobs = extract_jobs(last_page)
    return jobs

### `save.py`

In [20]:
# module import

import csv

# save.py

def save_to_file(jobs):
    file = open("jobs_Indeed.csv", mode="w", encoding="UTF-8") # 없으면 현재 경로에 파일 생성
    writer = csv.writer(file) # 방금 연 파일에 csv 작성
    writer.writerow(["title", "company", "location", "link"])
    for job in jobs:
        writer.writerow(list(job.values()))
    file.close()
    return

### `main.py`

In [21]:
indeed_jobs = get_jobs()
save_to_file(indeed_jobs)

Scrapping page 1
Scrapping page 2
Scrapping page 3
Scrapping page 4
Scrapping page 5
Scrapping page 6
Scrapping page 7
Scrapping page 8
Scrapping page 9
회사명이 없는 공고가 있습니다.
Scrapping page 10
회사명이 없는 공고가 있습니다.
회사명이 없는 공고가 있습니다.
Scrapping page 11
Scrapping page 12
Scrapping page 13
Scrapping page 14
회사명이 없는 공고가 있습니다.
Scrapping page 15
회사명이 없는 공고가 있습니다.
Scrapping page 16
Scrapping page 17
Scrapping page 18
회사명이 없는 공고가 있습니다.
회사명이 없는 공고가 있습니다.
Scrapping page 19
Scrapping page 20
